[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/Arif-PhyChem/trace_conservation/blob/main/physics_agnostic.ipynb)

In [3]:
import sys
import os

! git clone https://github.com/Arif-PhyChem/MLQD.git

Cloning into 'MLQD'...
remote: Enumerating objects: 1008, done.
remote: Counting objects: 100% (284/284), done.
remote: Compressing objects: 100% (190/190), done.
remote: Total 1008 (delta 135), reused 199 (delta 93), pack-reused 724
Receiving objects: 100% (1008/1008), 34.97 MiB | 24.85 MiB/s, done.
Resolving deltas: 100% (427/427), done.


In [4]:
# install the required packages

! pip install tensorflow
! pip install sklearn
! pip install hyperopt

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [6]:

mlqd_dir='MLQD/dev_ver'
sys.path.append(mlqd_dir)

from evolution import quant_dyn



In [7]:
# get data
! git clone https://github.com/Arif-PhyChem/trace_conservation.git

Cloning into 'trace_conservation'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 115 (delta 3), reused 36 (delta 3), pack-reused 79
Receiving objects: 100% (115/115), 324.69 MiB | 33.66 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Updating files: 100% (54/54), done.


In [15]:
! ls trace_conservation/training_data/

state_1  state_2


In [ ]:

param={
    'n_states': 2,
      'QDmodel': 'createQDmodel',     # str: create QD model. The dafault option is useQDmodel
      'QDmodelType': 'RCDYN',           # str: The type of model. Here KRR and the default option is OSTL
      'prepInput' : 'False',           # str: Prepare input files from the data (Default 'False')
      'XfileIn': 'trace_conservation/training_data/sb/physics_agnostic/state_1/x',           # str: (Optional) The prepared X file will be saved at the provided file name
      'YfileIn': 'trace_conservation/training_data/sb/physics_agnostic/state_1/y',           # str: (Optional) The prepared Y file will be saved at the provided file name
      'time': 10,
      'time_step': 0.05,
      'dataCol': 1,
      'xlength': 41,
      'hyperParam': 'False',           # str: Default is 'False', we can pass 'True' (optimize the hyperparameters) or 'False' (don't optimize and run with the default values)
      'patience': 30,
      'OptEpochs': 30,
      'TrEpochs': 500,
      'max_evals': 30,
      'systemType': 'SB', # str: (Not optional) Need to define, wether your model is spin-boson (SB) or FMO complex (FMO)
      'dataPath': '/home/dell/arif/pypackage/sb/data/training_data/sym',
      'QDmodelOut': 'RC_SB_model_1'    # str: (Optional), providing a name to save the model at
      }
quant_dyn(**param)



MLQD is a python package developed for Machine Learning-based Quantum Dissipative Dynamics,
 				  Version 1.1.1
			 https://github.com/Arif-PhyChem/MLQD

 			 Copyright (c) 2023 Arif Ullah

All rights reserved. This work is licensed under the Apache Software License 2.0


The above copyright notice and this permission notice shall be included 
in all copies or substantial portions of the Software.


The software is provided "as is" without warranty of any kind, express or implied, 
including but not limited to the warranties ofmerchantability, fitness for a particular 
purpose and noninfringement. In no event shall the authors or copyright holders be 
liable for any claim, damages or other liability, whether in an action of contract, tort or otherwise, 
arising from, out of or in connection with the software or the use or other dealings in the software.


				 Cite as:

1) Ullah A. and Dral P. O., Computer Physics Communications, 2023, 294, 108940
2) Ullah A. and Dral P. O., New Journ

In [ ]:
xx = np.load('/home/dell/arif/pypackage/sb/data/test_set/2_epsilon-0.0_Delta-1.0_lambda-0.2_gamma-10.0_beta-0.75.npy')


n_states = 2
xlength = 41
time = 10
time_step = 0.05
time_range=0
tt = time_range
for i in range(0, xlength + int(time/time_step)-1):
    tt += time_step
    time_range = np.append(time_range, tt)



model_1 = keras.models.load_model('state_1/RC_SB_model_0.keras') # state 1 CNN model
model_2 = keras.models.load_model('state_2/RC_SB_model_0.keras') # state 2 CNN model

models = [model_1, model_2]

x = np.zeros((xlength, n_states), dtype=float)
y = np.zeros((len(time_range), n_states), dtype=float)
yhat = np.zeros((1, n_states), dtype=float)

trace = np.zeros((len(time_range), 1), dtype=float)
labels = [1, 4]

i = 0
for label in labels:
    x[:,i] = np.real(xx[0:xlength, label])
    i += 1
y[0:xlength, :] = x[:,:]

for i in range (0, xlength):
  tr = 0
  for j in labels:
    tr += x[i, j]
  trace[i, 0] = tr

print('****************************************************************')
print('Predicting dynamics')
print('****************************************************************')

for i in range(xlength, len(time_range)):
    l =0
    for j in range (0, n_states):
        x_pred = x[:, j]
        x_pred = x_pred.reshape(1, x.shape[0], 1) # reshape the input
        yhat[0, j] =  models[l].predict(x_pred, verbose=0)[0][0]
        l += 1

    # correction
    trace = np.sum(yhat)

    x = np.delete(x, 0, 0)
    x = np.r_[x, yhat]
    y[i, :] = yhat[0, :]
np.save('tc_traj', np.c_['-1', time_range[:], y[:]])
print('Dynamics with no trace conservation is generated and saved in a file with name tc_traj.npy')